In [1]:
import os
import random
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib.cm as cm
from matplotlib import rcParams
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.isri import ISRIStemmer
from collections import Counter 
import itertools
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from joblib import dump, load
import pickle
import time as time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luche\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# data import 

In [2]:
import pandas as pd

df = pd.read_table('./data/twitter-2016test-A.txt' , usecols=[0,1,2], encoding='utf-8', names=['id','sentiment', 'tweet'])
#dataTest = pd.read_table('../input/semevalll/SemEval2017-test.txt', usecols=[1,2], encoding='utf-8', names=['sentiment', 'tweet'])                   
#combine = [dataTrain,dataTest]
df

,id,sentiment,tweet
0,619950566786113536,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T..."
1,619969366986235905,neutral,Order Go Set a Watchman in store or through ou...
2,619971047195045888,negative,If these runway renovations at the airport pre...
3,619974445185302528,neutral,If you could ask an onstage interview question...
4,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...
...,...,...,...
20627,681877834982232064,neutral,@ShaquilleHoNeal from what I think you're aski...
20628,681879579129200640,positive,"Iran ranks 1st in liver surgeries, Allah bless..."
20629,681883903259357184,neutral,Hours before he arrived in Saudi Arabia on Tue...
20630,681904976860327936,negative,@VanityFair Alex Kim Kardashian worth how to ...


# add label

In [3]:
CATEGORY_INDEX = {
    "negative": -1,
    "neutral": 0,
    "positive": 1
}

"""import data """
raw_label = df['sentiment'].values.tolist()
rawlabel = []
for i in range(len(raw_label)):
    rawlabel.append(CATEGORY_INDEX[raw_label[i]])
rawlabel

[0,
 0,
 -1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 -1,
 0,
 1,
 0,
 1,
 0,
 -1,
 0,
 1,
 0,
 0,
 0,
 -1,
 -1,
 1,
 0,
 1,
 -1,
 1,
 0,
 -1,
 -1,
 1,
 0,
 1,
 -1,
 -1,
 1,
 1,
 1,
 0,
 -1,
 0,
 0,
 0,
 -1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 -1,
 -1,
 0,
 -1,
 1,
 1,
 0,
 -1,
 0,
 1,
 1,
 -1,
 -1,
 -1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 -1,
 1,
 -1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 -1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 -1,
 1,
 1,
 0,
 1,
 1,
 -1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 -1,
 1,
 -1,
 1,
 -1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 -1,
 1,
 0,
 1,
 0,
 1,
 -1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 -1,
 0,
 -1,
 1,
 1,
 1,
 -1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 -1,
 1,
 1,
 1,
 -1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 -1,
 0

In [4]:
df['label'] = rawlabel
df

,id,sentiment,tweet,label
0,619950566786113536,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T...",0
1,619969366986235905,neutral,Order Go Set a Watchman in store or through ou...,0
2,619971047195045888,negative,If these runway renovations at the airport pre...,-1
3,619974445185302528,neutral,If you could ask an onstage interview question...,0
4,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...,1
...,...,...,...,...
20627,681877834982232064,neutral,@ShaquilleHoNeal from what I think you're aski...,0
20628,681879579129200640,positive,"Iran ranks 1st in liver surgeries, Allah bless...",1
20629,681883903259357184,neutral,Hours before he arrived in Saudi Arabia on Tue...,0
20630,681904976860327936,negative,@VanityFair Alex Kim Kardashian worth how to ...,-1


In [5]:
df_back = df.copy()
df_back

,id,sentiment,tweet,label
0,619950566786113536,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T...",0
1,619969366986235905,neutral,Order Go Set a Watchman in store or through ou...,0
2,619971047195045888,negative,If these runway renovations at the airport pre...,-1
3,619974445185302528,neutral,If you could ask an onstage interview question...,0
4,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...,1
...,...,...,...,...
20627,681877834982232064,neutral,@ShaquilleHoNeal from what I think you're aski...,0
20628,681879579129200640,positive,"Iran ranks 1st in liver surgeries, Allah bless...",1
20629,681883903259357184,neutral,Hours before he arrived in Saudi Arabia on Tue...,0
20630,681904976860327936,negative,@VanityFair Alex Kim Kardashian worth how to ...,-1


# data preprocessing

## 1 data cleansing 

In [6]:
df = df.dropna()

In [7]:
def clean_base(tweets, clean_object):
        #tweets.loc[:, "tweet"].replace(clean_object, "", inplace=True)
        tweets = re.sub(clean_object, ' ', tweets)
        return tweets

In [8]:
def remove_urls(tweets):
        return clean_base(tweets, re.compile(r"http.?://[^\s]+[\s]?"))

In [9]:
def remove_usernames(tweets):
        return clean_base(tweets, re.compile(r"@[^\s]+[\s]?"))

In [10]:
def remove_hashtags(tweets):  # it unrolls the hashtags to normal words
        for hashtag in map(lambda x: re.compile(re.escape(x)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                     "@", "%", "^", "*", "(", ")", "{", "}",
                                                                     "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                     "!", "?", ".", "'",
                                                                     "--", "---", "#"]):
            tweets = re.sub(hashtag, ' ', tweets)
        return tweets

In [11]:
def remove_numbers(tweets):
        return clean_base(tweets, re.compile(r"\s?[0-9]+\.?[0-9]*"))

In [12]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [13]:
##############################################

In [14]:
english_punctuations = string.punctuation
punctuations_list = english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [15]:
###############################################################

In [16]:
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = remove_usernames(doc)
    #Replace url with empty string
    doc = remove_urls(doc)

    
    #doc = re.sub(r'@[^\s]+', ' ', doc)
    #doc = re.sub(r'_', ' ', doc)
    
    
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('(www\.[^\s])',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    #doc= normalize_arabic(doc)
    
    #Replace numbers with empty string
    doc = remove_numbers(doc)
    #Replace @username with empty string
    doc = remove_hashtags(doc)
    # remove repeated letters
    #doc=remove_repeating_char(doc)
    
    #stemming
    doc = stemmer.stem(doc)
   
    return doc



stemmer = ISRIStemmer()
df["tweet"] = df['tweet'].apply(lambda x: processDocument(x, stemmer))
df

,id,sentiment,tweet,label
0,619950566786113536,neutral,Picturehouses Pink Floyds Roger Waters The Wal...,0
1,619969366986235905,neutral,Order Go Set a Watchman in store or through ou...,0
2,619971047195045888,negative,If these runway renovations at the airport pre...,-1
3,619974445185302528,neutral,If you could ask an onstage interview question...,0
4,619987808317407232,positive,A portion of book sales from our Harper LeeGo ...,1
...,...,...,...,...
20627,681877834982232064,neutral,from what I think youre asking in no order Fu...,0
20628,681879579129200640,positive,Iran ranks st in liver surgeries Allah bless t...,1
20629,681883903259357184,neutral,Hours before he arrived in Saudi Arabia on Tue...,0
20630,681904976860327936,negative,Alex Kim Kardashian worth how to love Kim Ka...,-1


In [98]:
#df.to_csv('./data.csv')

In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luche\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\luche\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [19]:
########################################trail 1#######################################################

In [20]:
data = []

# Separating our features (text) and our labels into two lists to smoothen our work
X = df['tweet'].tolist()
Y = df['label'].tolist()

# Building our data list, that is a list of tuples, where each tuple is a pair of the tokenized text
# and its corresponding label
for x, y in zip(X, Y):
    
    data.append((nltk.word_tokenize(x), y))
    
# Printing the CPU time and the first 5 elements of our 'data' list
#print('CPU Time:', time() - start_time)
data[:5]

[(['Picturehouses',
   'Pink',
   'Floyds',
   'Roger',
   'Waters',
   'The',
   'Walll',
   'opening',
   'Sept',
   'is',
   'now',
   'making',
   'waves',
   'Watch',
   'the',
   'trailer',
   'on',
   'Rolling',
   'Stone',
   'look'],
  0),
 (['Order',
   'Go',
   'Set',
   'a',
   'Watchman',
   'in',
   'store',
   'or',
   'through',
   'our',
   'website',
   'before',
   'Tuesday',
   'and',
   'get',
   'it',
   'half',
   'price',
   'GSAW'],
  0),
 (['If',
   'these',
   'runway',
   'renovations',
   'at',
   'the',
   'airport',
   'prevent',
   'me',
   'from',
   'seeing',
   'Taylor',
   'Swift',
   'on',
   'Monday',
   'Bad',
   'Blood',
   'will',
   'have',
   'a',
   'new',
   'meaning'],
  -1),
 (['If',
   'you',
   'could',
   'ask',
   'an',
   'onstage',
   'interview',
   'question',
   'at',
   'Miss',
   'USA',
   'tomorrow',
   'what',
   'would',
   'it',
   'be'],
  0),
 (['A',
   'portion',
   'of',
   'book',
   'sales',
   'from',
   'our',
   'Ha

In [21]:
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from time import time

In [22]:
pos_tag(data[0][0])

[('Picturehouses', 'NNS'),
 ('Pink', 'NNP'),
 ('Floyds', 'NNP'),
 ('Roger', 'NNP'),
 ('Waters', 'NNP'),
 ('The', 'DT'),
 ('Walll', 'NNP'),
 ('opening', 'NN'),
 ('Sept', 'NNP'),
 ('is', 'VBZ'),
 ('now', 'RB'),
 ('making', 'VBG'),
 ('waves', 'NNS'),
 ('Watch', 'VBP'),
 ('the', 'DT'),
 ('trailer', 'NN'),
 ('on', 'IN'),
 ('Rolling', 'NNP'),
 ('Stone', 'NNP'),
 ('look', 'NN')]

In [23]:
STOP_WORDS = stopwords.words('english')
def lemmatize_sentence(tweet_tokens):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        # Eliminating the token if it is a link
        
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token.lower(), pos)

        
        # Eliminating the token if its length is less than 3, if it is a punctuation or if it is a stopword
        if token not in string.punctuation and len(token) > 2 and token not in STOP_WORDS:
            cleaned_tokens.append(token)
            
    return cleaned_tokens
# Prevewing the remove_noise() output
print(lemmatize_sentence(data[0][0]))

['picturehouses', 'pink', 'floyds', 'roger', 'water', 'walll', 'opening', 'sept', 'make', 'wave', 'watch', 'trailer', 'rolling', 'stone', 'look']


In [24]:
start_time = time()

# As the Naive Bayesian classifier accepts inputs in a dict-like structure,
# we have to define a function that transforms our data into the required input structure
def list_to_dict(cleaned_tokens):
    return dict([token, True] for token in cleaned_tokens)

cleaned_tokens_list = []

# Removing noise from all the data
for tokens, label in data:
    cleaned_tokens_list.append((lemmatize_sentence(tokens), label))

print('Removed Noise, CPU Time:', time() - start_time)
start_time = time()

final_data = []

# Transforming the data to fit the input structure of the Naive Bayesian classifier
for tokens, label in cleaned_tokens_list:
    final_data.append((list_to_dict(tokens), label))
    
print('Data Prepared for model, CPU Time:', time() - start_time)

# Previewing our final (tokenized, cleaned and lemmatized) data list
final_data[:5]

Removed Noise, CPU Time: 65.58766412734985
Data Prepared for model, CPU Time: 0.08128595352172852


[({'picturehouses': True,
   'pink': True,
   'floyds': True,
   'roger': True,
   'water': True,
   'walll': True,
   'opening': True,
   'sept': True,
   'make': True,
   'wave': True,
   'watch': True,
   'trailer': True,
   'rolling': True,
   'stone': True,
   'look': True},
  0),
 ({'order': True,
   'set': True,
   'watchman': True,
   'store': True,
   'website': True,
   'tuesday': True,
   'get': True,
   'half': True,
   'price': True,
   'gsaw': True},
  0),
 ({'runway': True,
   'renovation': True,
   'airport': True,
   'prevent': True,
   'see': True,
   'taylor': True,
   'swift': True,
   'monday': True,
   'bad': True,
   'blood': True,
   'new': True,
   'meaning': True},
  -1),
 ({'could': True,
   'ask': True,
   'onstage': True,
   'interview': True,
   'question': True,
   'miss': True,
   'usa': True,
   'tomorrow': True,
   'would': True},
  0),
 ({'portion': True,
   'book': True,
   'sale': True,
   'harper': True,
   'leego': True,
   'set': True,
   'watchm

In [25]:
import pickle
pickle.dump(cleaned_tokens_list, open("./test.p" , "wb"))

In [3]:
import pickle

In [5]:
data_root = './test.p'

def load_data(root):
    """Load data from saved files generated in 'dataset_preprocess.py' """
    data = pickle.load(open(root, "rb"))
    return data

cleaned_tokens_list = load_data(data_root)

In [6]:
import numpy as np

In [26]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [27]:
# Loading the 50-dimensional GloVe embeddings
# This method will return three dictionaries:
# * word_to_index: a dictionary mapping from words to their indices in the vocabulary
# * index_to_word: dictionary mapping from indices to their corresponding words in the vocabulary
# * word_to_vec_map: dictionary mapping words to their GloVe vector representation
# Note that there are 400,001 words, with the valid indices ranging from 0 to 400,000

word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [28]:
def cosine_similarity(u, v):
    dot = np.dot(u, v)
    norm_u = np.sqrt(np.sum(u**2))
    norm_v = np.sqrt(np.sum(v**2))
    cosine_similarity = dot / (norm_u * norm_v)
    return cosine_similarity

In [29]:
#start_time = time()

unks = []
UNKS = []

# This function will act as a "last resort" in order to try and find the word
# in the words embedding layer. It will basically eliminate contiguously occuring
# instances of a similar character
def cleared(word):
    res = ""
    prev = None
    for char in word:
        if char == prev: continue
        prev = char
        res += char
    return res


def sentence_to_indices(sentence_words, word_to_index, max_len, i):
    global X, Y
    sentence_indices = []
    for j, w in enumerate(sentence_words):
        try:
            index = word_to_index[w]
        except:
            UNKS.append(w)
            w = cleared(w)
            try:
                index = word_to_index[w]
            except:
                index = word_to_index['unk']
                unks.append(w)
        X[i, j] = index


# Here we will utilize the already computed 'cleaned_tokens_list' variable
   
#print('Removed Noise, CPU Time:', time() - start_time)
#start_time = time()

list_len = [len(i) for i, j in cleaned_tokens_list]
max_len = max(list_len)
print('max_len:', max_len)

X = np.zeros((len(cleaned_tokens_list), max_len))
Y = np.zeros((len(cleaned_tokens_list), ))

for i, tk_lb in enumerate(cleaned_tokens_list):
    tokens, label = tk_lb
    sentence_to_indices(tokens, word_to_index, max_len, i)
    Y[i] = label
    
#print('Data Prepared for model, CPU Time:', time() - start_time)


print(X[:100])
print(Y[:100])

max_len: 22
[[372306. 285068. 150438. ...      0.      0.      0.]
 [271173. 325899. 383760. ...      0.      0.      0.]
 [313470. 305627.  49231. ...      0.      0.      0.]
 ...
 [111219. 192758. 230972. ...      0.      0.      0.]
 [372306. 246253. 374068. ...      0.      0.      0.]
 [269331. 198755. 389401. ...      0.      0.      0.]]
[ 0.  0. -1.  0.  1.  1.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  1.  0.  1.  1.  0.  1.  0. -1.  0.  1.  0.  1.  0. -1.  0.  1.
  0.  0.  0. -1. -1.  1.  0.  1. -1.  1.  0. -1. -1.  1.  0.  1. -1. -1.
  1.  1.  1.  0. -1.  0.  0.  0. -1.  1.  0.  1.  0.  1.  0.  1.  1.  0.
  0.  1. -1. -1.  0. -1.  1.  1.  0. -1.  0.  1.  1. -1. -1. -1.  0.  1.
  0.  1.  0.  0.  0.  0.  1.  1.  0.  1.]


In [30]:
from tensorflow.keras.utils import to_categorical
Y = to_categorical(Y,3)
Y

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

In [31]:
import pickle
pickle.dump(X, open("./test_X.p" , "wb"))
pickle.dump(Y, open("./test_Y.p" , "wb"))

In [19]:
import keras
from keras import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Bidirectional,GlobalMaxPool1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [20]:
my_seed = 22
np.random.seed(my_seed)

In [21]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index, max_len):
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["unk"].shape[0] #50
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]
        
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False, input_shape=(max_len,))
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [32]:
model = Sequential()

model.add(pretrained_embedding_layer(word_to_vec_map, word_to_index, max_len))

# Apply Dropout to use recurrent decimation to combat overfitting 
model.add(LSTM(128, return_sequences=True, dropout=0.4, recurrent_dropout=0.4)) 

# Use Flatten() or GlobalMaxPooling1D() to convert 3D output to 2D.
# It allows you to add one or more Dense layers to your model 
model.add(GlobalMaxPool1D())

#model.add(Dense(64, activation='relu'))#as experiment we tried to add one additional layer,but it is not give as higher accuracy
#model.add(Dropout(0.4))

model.add(Dense(3, activation='softmax'))#activation function for classification
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 536, 50)           20000050  
_________________________________________________________________
lstm_3 (LSTM)                (None, 536, 128)          91648     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 20,092,085
Trainable params: 92,035
Non-trainable params: 20,000,050
_________________________________________________________________


In [ ]:
model.load_weights('./data/weights.best.1.hdf5')
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])
print('Created model and loaded weights from hdf5 file')
 
# estimate
scores = model.evaluate(X,Y)
print("{0}: {1:.2f}%".format(model.metrics_names[1], scores[1]*100))